In [20]:
import uuid
import os
import tempfile

import ffmpeg
# NOTE: need to make sure ffmpeg cli is installed

In [26]:
def make_video_clip(input_video_filename, output_video_filename, start_time, end_time):
    ffmpeg.input(input_video_filename, ss=start_time, to=end_time).output(output_video_filename).run(overwrite_output=True)

In [51]:
def combine_timestamps(input_video_filename, output_video_filename, timestamp_windows):
    # create clips of video for timestamp windows and save to temp dir
    # combine clips into video
    with tempfile.TemporaryDirectory() as tempdir:
        # generate clips from source video by time stamp window
        outfiles = []
        
        for window in timestamp_windows:
            out = f"{tempdir}/{str(uuid.uuid4())[:8]}.mp4"

            make_video_clip(input_video_filename, out, window[0], window[1])
            outfiles.append(f"file {out}")

        # Combine outfile paths into a txt file
        combined_file = f"{tempdir}/{str(uuid.uuid4())[:8]}.txt"

        with open(combined_file, "w") as fp:
            fp.write("\n".join(outfiles))

        # Combine clips using source files
        ffmpeg.input(combined_file, format='concat', safe=0).output(output_video_filename, c='copy').run(overwrite_output=True)

In [53]:
start_time_1 = '00:00:00' # Start time for trimming (HH:MM:SS)
end_time_1 = '00:00:5' # End time for trimming (HH:MM:SS)
start_time_2 = '00:00:10' # Start time for trimming (HH:MM:SS)
end_time_2 = '00:00:15' # End time for trimming (HH:MM:SS)

combine_timestamps("random/m2-res_640p.mp4", "random/out.mp4", [(start_time_1, end_time_1), (start_time_2, end_time_2)])

ffmpeg version 6.1.1 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/6.1.1_7 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libopenvino --enable-li